In [ ]:
import tensorflow as tf
print(tf.__version__)

[slides](https://github.com/arthurredfern/UT-Dallas-CS-6301-CNNs/blob/master/Lectures/xNNs_070_Implementation.pdf)

[disk](file:///F:/Data/xNNs_070_Implementation.pdf)

# Implementation Notes

## Hardware and Software

Software can be more general (hardware agnostic) that leans on runtime intelligent hardware to optimize the perforance of the program duting execution. 

Software can rely on compile time intelligent hardware. 

__Graph Specification__ Is a representation of a high-level hardware agnostic computer algorithm. nodes are operators. Implicit data and instruction movement. Edges prepresent dependenies (memory)

__Graph Lowering__ Map a high level program to a low-level, hardware specific graph. Includes compute and communication nodes, as well as data and instruction edges. Nodes map 1 to 1 to hardware.

This is an iterative optimization process that can be improved with knowledge of domain and hardware.

__Graph Execution__ Sofftware runtime runs on a control processor and cycles through nodes on the low level graph. Hardware execures the nodes with computation and communication primitives as well as general compute for everything else.

## Optimized XNN Design

Some hardware aware design considerations:
- Operator selection: quantization, sparisification, and compression
- Network sizing: depth, width, input size

__Quantization__ reduces the number of bits required to represent feature maps and parameters. 

Quantizing deep convolutional networks for efficient inference: A whitepaper
https://arxiv.org/abs/1806.08342

*Per-channel quantization of weights and per-layer quantization of activations to 8-bits of precision post-training produces classification accuracies within 2% of floating point networks for a wide variety of CNN architectures. Model sizes can be reduced by a factor of 4 by quantizing weights to 8-bits, even when 8-bit arithmetic is not supported...Observe a speedup of 2x-3x for quantized implementations compared to floating point on CPUs*

*We recommend that per-channel quantization of weights and per-layer quantization of activations be the preferred quantization scheme for hardware acceleration and kernel optimization. We also propose that future processors and hardware accelerators for optimized inference support precisions of 4, 8 and 16 bits.*

Reduces memory and communication requirements of the network. Multiplication complexity scales to the square of the number of bits, while addition scales linearly with the number of bits.

32 bit float, 16 bit float, 8 bit fixed (common for CNN inference), and 4 bit fixed

__Sparsification__

Reduces memory and the required number of computations. Random sparsity can be introduced with L1 regularization as well as thresholding dense coefficient values.

Network pruning methods exist. Using second oreder derivatives of parameters wrt error function

https://openreview.net/pdf?id=Sy1iIDkPM

## XNN Software

__CNN Graph Lowering__

• Domain agnostic hardware agnostic optimization
    - Remove unneeded edges and nodes required for specified input output
    - Constant folding and constant propagation
    
• Domain specific hardware agnostic optimization
    - xNNs: remove dropout and scale associated weight layers
    - xNNs: absorb batch norm into convolution and create a bias term

• Domain specific hardware aware optimization
    - xNNs: transform data layouts (tensor ordering)
    - xNNs: node fusion, tiling and grouping
    - xNNs: post training quantization

• Domain agnostic hardware specific code generation
    - Memory planning for all tensors
    - Data movement and compute strategy selection for each node
    - Code generation for selected strategy

## XNN Performance